In [ ]:
import os
import gc
import pandas as pd
import numpy as np
import shutil
import torch
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
import json
import re

from torch.utils.data import WeightedRandomSampler
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, EarlyStoppingCallback, AutoConfig
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
from torch.nn import CrossEntropyLoss
from huggingface_hub import create_repo, upload_folder, notebook_login
from utils_dl import set_global_seed

try: 
    from sklearn.model_selection import StratifiedGroupKFold
    GROUP_SPLITTER = StratifiedGroupKFold(n_splits=8,
                                          shuffle=True,
                                          random_state=42)
except ImportError as e:
    print('ImportError:', e)
    GROUP_SPLITTER = None  # fallback later

torch.cuda.empty_cache()

In [ ]:
SEED=42
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SUBTASK_1_PATH = "new_data\subtask1"

set_global_seed(SEED)
notebook_login() # Log into HF account

In [ ]:
# Configuration parameters
language = 'spa'
model_type = 'dl'
stemming = False
lemmatization = False
remove_duplicates = False
cased = True 

data_config = f"lang_{language}_model_{model_type}_stem_{stemming}_lem_{lemmatization}_dup_{remove_duplicates}_cased_{cased}"
file_name = 'subtask1_all_aug'
db_file_name = f"{file_name}_{data_config}.csv"
file_path = os.path.join(SUBTASK_1_PATH, db_file_name)

if os.path.exists(file_path):
    full_data = pd.read_csv(file_path, encoding='utf-8')
    # print("File found:")
    # print(full_data.info())
else:
    raise FileNotFoundError(f"File not found at {file_path}")

In [ ]:
# For augmented data: samples sharing the same 'id' belong to the same original instance.
full_data["id"] = full_data["id"].astype(str)
full_data = full_data[full_data['augmentation_type'] != 'synonym_replacement'] 
full_data = full_data[full_data['augmentation_type'] != 'aeda_2'] # drop aeda_2 augmentation we use aeda_1

In [ ]:
text_column  = "lyrics_clean"
label_column = "label"
group_column = "id"             # all augmented variants share this id
aug_col      = "is_augmented"   # bool
final_training = False 

# labels
if full_data[label_column].dtype == object:
    unique_labels = sorted(full_data[label_column].unique())
    label2id = {lbl: idx for idx, lbl in enumerate(unique_labels)}
else:
    unique_labels = sorted(full_data[label_column].unique())
    label2id = {int(lbl): int(lbl) for lbl in unique_labels}

id2label = {v: k for k, v in label2id.items()}
full_data[label_column] = full_data[label_column].map(label2id)
    
# Splits
full_data = full_data.sample(frac = 1, random_state=SEED) 
splits_folder_path =  os.path.join(SUBTASK_1_PATH, "splits") 


if not final_training:

    train_file_name = 'train_ids.csv'
    test_file_name = 'test_ids.csv'
    val_file_name = 'val_ids.csv'
    
    train_file_path = os.path.join(splits_folder_path, train_file_name)
    test_file_path = os.path.join(splits_folder_path, test_file_name)
    val_file_path = os.path.join(splits_folder_path, val_file_name)
    
    # Get train ids
    if os.path.exists(train_file_path):
        unique_ids_df = pd.read_csv(train_file_path, encoding='utf-8')
        # Select rows in full_data whose 'id' is in unique_ids
        train_df = full_data[full_data['id'].isin(unique_ids_df['id'])].reset_index(drop=True)
    else:
        raise FileNotFoundError(f"File not found at {test_file_path}")
    
    if os.path.exists(test_file_path):
        unique_ids_df = pd.read_csv(test_file_path, encoding='utf-8')
        # Select rows in full_data whose 'id' is in unique_ids
        test_df = full_data[full_data['id'].isin(unique_ids_df['id'])].reset_index(drop=True)
    else:
        raise FileNotFoundError(f"File not found at {test_file_path}")
        
    if os.path.exists(val_file_path):
        unique_ids_df = pd.read_csv(val_file_path, encoding='utf-8')
        # Select rows in full_data whose 'id' is in unique_ids
        val_df = full_data[full_data['id'].isin(unique_ids_df['id'])].reset_index(drop=True)
    else:
        raise FileNotFoundError(f"File not found at {val_file_path}")
    
else:
    train_file_name = 'train_competition_ids.csv'
    val_file_name = 'val_competition_ids.csv'
    
    train_file_path = os.path.join(splits_folder_path, train_file_name)
    val_file_path = os.path.join(splits_folder_path, val_file_name)
    
    # Get train ids
    if os.path.exists(train_file_path):
        unique_ids_df = pd.read_csv(train_file_path, encoding='utf-8')
        # Select rows in full_data whose 'id' is in unique_ids
        train_df = full_data[full_data['id'].isin(unique_ids_df['id'])].reset_index(drop=True)
    else:
        raise FileNotFoundError(f"File not found at {train_file_path}")
        
    if os.path.exists(val_file_path):
        unique_ids_df = pd.read_csv(val_file_path, encoding='utf-8')
        # Select rows in full_data whose 'id' is in unique_ids
        val_df = full_data[full_data['id'].isin(unique_ids_df['id'])].reset_index(drop=True)
    else:
        raise FileNotFoundError(f"File not found at {val_file_path}")
    

In [ ]:
# Remove augmented data from val / test
if not final_training:
    # *Train / Val / Test**
    val_df  = val_df[val_df[aug_col]  != True]
    test_df = test_df[test_df[aug_col] != True]
    # print(f"train={len(train_df)},  val={len(val_df)},  test={len(test_df)}")
else:
    val_df  = val_df[val_df[aug_col]  != True]
    # print(f"train={len(train_df)}, val={len(val_df)}")

In [ ]:
if not final_training: 

    train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
    val_dataset  = Dataset.from_pandas(val_df.reset_index(drop=True))
    test_dataset  = Dataset.from_pandas(test_df.reset_index(drop=True))
    
    ds = DatasetDict({
        'train': train_dataset,
        'val': val_dataset,
        'test': test_dataset
    })
   
    
else: # **Train / Val**
    
    train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
    val_dataset  = Dataset.from_pandas(val_df.reset_index(drop=True))
    
    ds = DatasetDict({
        'train': train_dataset,
        'val': val_dataset
    })
    
# print(ds)

## Text Representation

In [ ]:
MODEL_CHECKPOINT = "PlanTL-GOB-ES/longformer-base-4096-bne-es"
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT, use_fast=True) 

In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples[text_column],
        padding=False,  # dynamic later
        truncation=True    
)

tokenized_ds = ds.map(tokenize_function, batched=True)


columns_to_keep = {"input_ids", "attention_mask", label_column}
for split in tokenized_ds.keys():    
    tokenized_ds[split] = tokenized_ds[split].remove_columns([col for col in tokenized_ds[split].column_names if col not in columns_to_keep])
    tokenized_ds[split] = tokenized_ds[split].rename_column(label_column, "labels")
    tokenized_ds[split].set_format("torch")

data_collator = DataCollatorWithPadding(tokenizer,
                                        pad_to_multiple_of=8,   # faster on tensor‑cores
                                        return_tensors="pt")

## Model Learning (HPO - Optuna)

In [ ]:

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [ ]:
# Compute class weights for imbalanced data
# Weighted loss: keep your data distribution but penalize errors on minority classes more heavily
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.arange(len(unique_labels)),
    y=train_df[label_column].to_numpy()
)
weight_tensor = torch.tensor(class_weights, dtype=torch.float)
# print("Class weights:", weight_tensor)

# Balanced sampling: over‑ or under‑sample examples so that each batch is roughly class‑balanced
# sampling probabilities
sample_weights = class_weights[train_df[label_column].to_numpy()]
sampler = WeightedRandomSampler(weights=sample_weights,
                                num_samples=len(sample_weights),
                                replacement=True)


class BalancedTrainer(Trainer):
    def __init__(self, *args, weight_tensor: torch.Tensor = None, **kwargs):
        super().__init__(*args, **kwargs)
        if weight_tensor is None:
            raise ValueError("You must pass weight_tensor")
        self.weight_tensor = weight_tensor

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # pop labels & forward
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        weight = self.weight_tensor.to(model.device)
        loss_fct = CrossEntropyLoss(weight=weight)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

    # balanced batches 
    def get_train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.train_dataset,
            batch_size=self.args.per_device_train_batch_size,
            sampler=sampler,
            collate_fn=self.data_collator,
            drop_last=False,
            num_workers=4,
            pin_memory=True,
        )

In [26]:
num_labels = len(unique_labels)
optuna_path = "./hpo_{}".format(MODEL_CHECKPOINT.replace("/","_"))
if os.path.exists(optuna_path):
    shutil.rmtree(optuna_path)

In [ ]:
def objective(trial):
    # HPO
    lr     = trial.suggest_float("learning_rate", 1e-6, 5e-5, log=True)
    wd     = trial.suggest_float("weight_decay", 0.0, 0.3)
    bsz    = trial.suggest_categorical("batch_size", [6])
    drop_h = trial.suggest_float("hidden_dropout", 0.1, 0.4)
    drop_a = trial.suggest_float("attn_dropout", 0.1, 0.4)
    drop_c = trial.suggest_float("classifier_dropout", 0.1, 0.4)

    # model_init used by Trainer
    def model_init():
        cfg = AutoConfig.from_pretrained(
          MODEL_CHECKPOINT,
          num_labels=len(unique_labels),
          label2id=label2id,
          id2label=id2label,
          hidden_dropout_prob=drop_h,
          attention_probs_dropout_prob=drop_a,
          classifier_dropout=drop_c,
        )
        return AutoModelForSequenceClassification.from_pretrained(
          MODEL_CHECKPOINT, config=cfg
        )

    # Training args
    args = TrainingArguments(
      output_dir                 = os.path.join(optuna_path, f"trial_{trial.number}"),
      eval_strategy              = "steps",
      eval_steps                 = 100,
      logging_strategy           = "steps",
      logging_steps              = 100,
      save_strategy              = "steps",
      save_steps                 = 100,
      save_total_limit           = 1,
      load_best_model_at_end     = True,
      metric_for_best_model      = "eval_f1",
      gradient_accumulation_steps = max(1, 32 // bsz), 
      greater_is_better          = True,
      learning_rate              = lr,
      weight_decay               = wd,
      per_device_train_batch_size= bsz,
      per_device_eval_batch_size = bsz,
      num_train_epochs           = 12,
      fp16                       = True,
      seed                       = SEED,
      report_to                  = "none",
    )

    # Trainer with early stop
    trainer = BalancedTrainer(
      model_init     = model_init,
      args           = args,
      train_dataset  = tokenized_ds["train"],
      eval_dataset   = tokenized_ds["val"],
      tokenizer      = tokenizer,
      data_collator  = data_collator,
      compute_metrics= compute_metrics,
      weight_tensor  = weight_tensor,        # precomputed outside
      callbacks      = [
        EarlyStoppingCallback(early_stopping_patience=4),
      ],
    )

    trainer.train()
    metrics = trainer.evaluate()
    gc.collect(); torch.cuda.empty_cache()
    return metrics["eval_f1"]

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

#### Save model in Hugging Face Hub

In [ ]:

best_trial = study.best_trial.number
trial_dir = os.path.join(optuna_path, f"trial_{best_trial}")

state_fp = os.path.join(trial_dir, "trainer_state.json")

if not os.path.isfile(state_fp):
    pat  = re.compile(r"checkpoint-(\d+)$")
    ckpt_dirs = [d for d in os.listdir(trial_dir) if pat.match(d)]
    if not ckpt_dirs:
        raise FileNotFoundError(f"No checkpoints found in {trial_dir}")

    newest = max(ckpt_dirs, key=lambda d: int(pat.match(d).group(1)))
    state_fp = os.path.join(trial_dir, newest, "trainer_state.json")

    if not os.path.isfile(state_fp):
        raise FileNotFoundError("trainer_state.json not found even in checkpoint.")

with open(state_fp) as f:
    j = json.load(f)
    best_ckpt_path = j["best_model_checkpoint"]
    best_epoch = j["epoch"]

print("Best checkpoint:", best_ckpt_path)

Best checkpoint: ./hpo_PlanTL-GOB-ES_longformer-base-4096-bne-es/trial_5/checkpoint-800


In [29]:
model     = AutoModelForSequenceClassification.from_pretrained(best_ckpt_path)
tokenizer = AutoTokenizer.from_pretrained(best_ckpt_path)
best_params = study.best_trial.params 
best_params['epoch'] = best_epoch

repo_id = f"SeTo97/{MODEL_CHECKPOINT.split('/')[-1]}_ft_70_2"
create_repo(repo_id, private=True, repo_type="model", exist_ok=True)

upload_folder(
    repo_id        = repo_id,
    folder_path    = best_ckpt_path,        # model + tokenizer together
    repo_type      = "model",
    commit_message = (
        f"Best F1={study.best_trial.value:.4f} "
        f"(trial {study.best_trial.number})\n"
        f"Hyper‑parameters: {best_params}"
    ),
)

scaler.pt:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

optimizer.pt:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

scheduler.pt:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

rng_state.pth:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/595M [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.78k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/SeTo97/longformer-base-4096-bne-es_ft_70_2/commit/3aac173d8ee243687e60f2e0c77db22ee1285722', commit_message="Best F1=0.7937 (trial 5)\nHyper‑parameters: {'learning_rate': 1.1990640966033497e-05, 'weight_decay': 0.02779141111933888, 'batch_size': 6, 'hidden_dropout': 0.14305022315729415, 'attn_dropout': 0.2149871315628947, 'classifier_dropout': 0.1312020858420528, 'epoch': 3.861405197305101}", commit_description='', oid='3aac173d8ee243687e60f2e0c77db22ee1285722', pr_url=None, repo_url=RepoUrl('https://huggingface.co/SeTo97/longformer-base-4096-bne-es_ft_70_2', endpoint='https://huggingface.co', repo_type='model', repo_id='SeTo97/longformer-base-4096-bne-es_ft_70_2'), pr_revision=None, pr_num=None)

## External Evaluation (held-out test set)

In [ ]:
model_repo = repo_id

model = AutoModelForSequenceClassification.from_pretrained(model_repo)
tokenizer = AutoTokenizer.from_pretrained(model_repo)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

test_dataset = tokenized_ds["test"]


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator  
)

results = trainer.evaluate(test_dataset)

print("Test Set Evaluation:")
for metric, value in results.items():
    print(f"{metric}: {value:.4f}")


/tmp/ipykernel_1037/124484822.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Test Set Evaluation:
eval_loss: 0.6236
eval_model_preparation_time: 0.0020
eval_accuracy: 0.8157
eval_precision: 0.8038
eval_recall: 0.7911
eval_f1: 0.7964
eval_runtime: 10.3347
eval_samples_per_second: 43.0590
eval_steps_per_second: 5.4190
